In [22]:
!pip install torch

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [34]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask = None):
        # take dot product of Q and K transpose
        attn_scores = torch.matmul(Q, K.transpose(-1, -2)) / math.sqrt(self.d_k)
        
        # apply mask
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # pass through softmax layer
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # take dot product of attn_probs and V
        output = torch.matmul(attn_probs, V)
        return output
    
    """
    Reshaping input tensors:
        input: [batch_size x seq_length x d_model]
        output: [batch_size x num_heads x seq_length x dim_per_head] 
    """
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
    
    """
    Reshaping input tensors:
        input: [batch_size x num_heads x seq_length x dim_per_head] 
        output: [batch_size x seq_length x d_model]
    """
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
    
    def forward(self, Q, K, V, mask = None):
        # reshape input tensors
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # dot product
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # combine heads
        output = self.W_o(self.combine_heads(attn_output))
        
        return output
    

In [35]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [36]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0)/d_model))
            
        # Apply sin to even columns and cos to odd columns of pe
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1:: 2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]
        

In [37]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        # pass through attention block
        attn_output = self.self_attn(x, x, x, mask)
        
        # add and norm
        x = self.norm1(x + self.dropout(attn_output))
        
        # pass through feed forward layer
        ff_output = self.feed_forward(x)
        
        # add and norm
        x = self.norm2( x + self.dropout(ff_output))
        
        return x

In [38]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.norm2 = nn.LayerNorm(d_model)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        # self attention of decoder input
        attn_output = self.self_attn(x, x, x, tgt_mask)
        
        # add and norm
        x = self.norm1(x + self.dropout(attn_output))

        # cross attention
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
            
        # add and norm
        x = self.norm2(x + self.dropout(attn_output))

        # pass through feed forward layer
        ff_output = self.feed_forward(x)

        # add and norm
        x = self.norm3(x + self.dropout(ff_output))
        
        return x
    

In [39]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, 
                 num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask
        
    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        
        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)
            
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
            
        output = self.fc(dec_output)
        return output
        

In [40]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))

In [42]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0005, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 4.921694755554199
Epoch: 2, Loss: 8.323627471923828
Epoch: 3, Loss: 7.25517463684082
Epoch: 4, Loss: 7.616508483886719
Epoch: 5, Loss: 7.458904266357422
Epoch: 6, Loss: 6.69974422454834
Epoch: 7, Loss: 5.864558696746826
Epoch: 8, Loss: 5.845470905303955
Epoch: 9, Loss: 5.487013816833496
Epoch: 10, Loss: 5.34512996673584
Epoch: 11, Loss: 4.949604511260986
Epoch: 12, Loss: 4.769575119018555
Epoch: 13, Loss: 4.642838001251221
Epoch: 14, Loss: 4.386444568634033
Epoch: 15, Loss: 4.1618146896362305
Epoch: 16, Loss: 3.967585802078247
Epoch: 17, Loss: 3.75949764251709
Epoch: 18, Loss: 3.56465744972229
Epoch: 19, Loss: 3.3954079151153564
Epoch: 20, Loss: 3.2228260040283203
Epoch: 21, Loss: 3.0450968742370605
Epoch: 22, Loss: 2.8847126960754395
Epoch: 23, Loss: 2.7506606578826904
Epoch: 24, Loss: 2.6438610553741455
Epoch: 25, Loss: 2.5295629501342773
Epoch: 26, Loss: 2.3643054962158203
Epoch: 27, Loss: 2.2039811611175537
Epoch: 28, Loss: 2.0622951984405518
Epoch: 29, Loss: 1.9198

KeyboardInterrupt: 